In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from  keras.models import Sequential, Model
from  keras.layers import Input, Dense, LeakyReLU, Activation, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from  keras.applications.vgg16 import VGG16, preprocess_input
#from  keras.applications import ResNet152V2,ResNet101V2,ResNet50V2 
#from  keras.applications.resnet_v2 import preprocess_input 

from  keras.optimizers import RMSprop, Adam
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [8]:
BATCH_SIZE = 32
IMG_SIZE = 224

In [9]:
def yolf_loss(y_true, y_pred):
    #true e pred sono [32][245]
    
    #tensorflow.print("tensors:", y_pred)
    #tensorflow.print("batch_size:", tensorflow.shape(y_pred)[0])
    
    y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    y_true = tensorflow.reshape(y_true, [tensorflow.shape(y_pred)[0], 7, 7, 5])


    b_p_pred = y_pred[..., 0]
    b_x_pred = y_pred[..., 1]
    b_y_pred = y_pred[..., 2]
    b_w_pred = y_pred[..., 3]
    b_h_pred = y_pred[..., 4]


    b_p = y_true[..., 0]
    b_x = y_true[..., 1]
    b_y = y_true[..., 2]
    b_w = y_true[..., 3]
    b_h = y_true[..., 4]

    #print(b_xy_pred.get_shape(),b_xy.get_shape())
    #print(b_wh_pred.get_shape(),b_wh.get_shape())
    #indicator_coord = K.expand_dims(y_true[..., 3], axis=-1) * 1.0
    loss_p = K.sum(K.square(b_p - b_p_pred), axis=-1)
    loss_xy = K.sum(b_p * (K.square(b_x - b_x_pred) + K.square(b_y - b_y_pred)), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])
    
    b_w = K.pow(b_w, 0.5)
    b_h = K.pow(b_h, 0.5)
    b_w_pred = K.pow(b_w_pred, 0.5)
    b_h_pred = K.pow(b_h_pred, 0.5)
    
    loss_wh = K.sum(
        b_p * 
        (
            (K.square(b_w - b_w_pred)) + 
            (K.square(b_h - b_h_pred))
        ), axis=-1)# * indicator_coord)#, axis=[1,2,3,4])

    #tensorflow.print("loss_p:", loss_p)
    #tensorflow.print("loss_xy:", loss_xy)
    #tensorflow.print("loss_wh:", loss_wh)

    #print(K.cast(loss_p, dtype="float32"))
    #print(K.cast(loss_xy, dtype="float32"))
    #print(loss_wh)
    #tensorflow.print("Loss:", ( K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32") )/3)
    return (K.cast(loss_p, dtype="float32") + loss_wh +  K.cast(loss_xy, dtype="float32"))/3

In [10]:
train_set = "./dataset/NUOVO/train/"
validation_set = "./dataset/NUOVO/val/"
test_set = "./dataset/NUOVO/test/"

In [11]:
train_df = pandas.read_csv("./dataset/NUOVO/train/output_segmentationv2.csv")
valid_df = pandas.read_csv("./dataset/NUOVO/val/output_segmentationv2.csv")
test_df = pandas.read_csv("./dataset/NUOVO/test/output_segmentationv2.csv")

In [12]:
valid_df.columns[1:]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '235', '236', '237', '238', '239', '240', '241', '242', '243', '244'],
      dtype='object', length=245)

In [13]:
train_datagen = ImageDataGenerator(
    #rescale=1./255,
    preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=train_set,
        x_col="image_id",
        y_col=train_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=validation_set,
        x_col="image_id",
        y_col=valid_df.columns[1:],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [15]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

In [16]:
res = VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [17]:
res.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [31]:
model = Sequential()
model.add(res)
model.add(Conv2D(1024, 3, strides=(1, 1), padding='same',activation="relu"))
model.add(Conv2D(1024, 3, strides=(1, 1), padding='same',activation="relu"))
model.add(Conv2D(1024, 3, strides=(1, 1), padding='same',activation="relu"))
model.add(Conv2D(5, 1,activation="relu"))
model.add(Flatten())
#model.add(Dense(512,activation="relu"))
#model.add(Dense(512,activation="relu"))
#model.add(LeakyReLU(alpha=0.3))
#model.add(Dropout(0.5))
#model.add(Dense(245, activation="relu"))
#model.add(Dropout(0.5))

In [32]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 7, 7, 1024)        4719616   
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 7, 7, 1024)        9438208   
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 7, 7, 1024)        9438208   
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 7, 7, 5)           5125      
_________________________________________________________________
flatten_1 (Flatten)          (None, 245)               0         
Total params: 38,315,845
Trainable params: 38,315,845
Non-trainable params: 0
__________________________________________

model = multi_gpu_model(model, gpus=2) #parallelize model

In [33]:
model.compile(optimizer=Adam(lr = 1e-4), loss="mse", metrics=["mse"])

In [34]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)


570/569 [==============================] - 53s 93ms/step


[204.38963317871094, 190.13023376464844]

In [36]:
#earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=20, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, reduce_lr_loss])

Epoch 1/20
5129/5128 [==============================] - 1461s 285ms/step - loss: 87.2434 - mse: 87.2390 - val_loss: 83.0533 - val_mse: 63.9454
Epoch 2/20
5129/5128 [==============================] - 1459s 284ms/step - loss: 61.3403 - mse: 61.3441 - val_loss: 50.6551 - val_mse: 62.1227
Epoch 3/20
5129/5128 [==============================] - 1461s 285ms/step - loss: 59.2473 - mse: 59.2397 - val_loss: 49.7983 - val_mse: 59.8692
Epoch 4/20
5129/5128 [==============================] - 1461s 285ms/step - loss: 57.2706 - mse: 57.2632 - val_loss: 50.0112 - val_mse: 59.4939
Epoch 5/20
5129/5128 [==============================] - 1457s 284ms/step - loss: 55.6562 - mse: 55.6567 - val_loss: 76.7481 - val_mse: 59.9943
Epoch 6/20
 898/5128 [====>.........................] - ETA: 19:21 - loss: 53.9217 - mse: 53.9054

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training accuracy')
plt.plot(val_acc, label='Validation accuracy')
plt.legend(loc='lower right')
plt.ylabel('accuracy')
plt.ylim([min(plt.ylim()),500])
plt.title('Training and Validation accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,20000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

In [18]:
model_json = model.to_json()
with open("yolf_vgg.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("resnet50.h5")
print("Saved model to disk")

Saved model to disk


# Restart training from here

In [11]:
import keras
json_file = open('yolf_vgg.json', 'r') 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = keras.models.model_from_json(loaded_model_json) 
loaded_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 1024)        4719616   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 1024)        9438208   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 1024)        9438208   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              51381248  
_________________________________________________________________
dense_2 (Dense)              (None, 512)             

In [12]:
#load weights into new model 
loaded_model.load_weights("yolf.h5") 
print("Loaded model from disk") 
#evaluate loaded model on test data 
loaded_model.compile(optimizer= keras.optimizers.Adam(lr=1e-4), loss = 'mse', metrics=['mse'])

Loaded model from disk


In [13]:
loaded_model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)


570/569 [==============================] - 56s 99ms/step


[204.91500854492188, 227.0862579345703]

In [16]:
#earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint('yolf.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min')

history = loaded_model.fit_generator(generator=train_generator, epochs=20, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, reduce_lr_loss])

Epoch 1/20
5129/5128 [==============================] - 1584s 309ms/step - loss: 154.6444 - mse: 154.6603 - val_loss: 283.6962 - val_mse: 235.0415
Epoch 2/20
 628/5128 [==>...........................] - ETA: 22:23 - loss: 149.5092 - mse: 149.5092

KeyboardInterrupt: 

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_set,
                                    x_col="image_id",
                                    y_col=["x_1","y_1","width","height"],
                                    class_mode="raw",
                                    shuffle=True,
                                    target_size=(224, 224),
                                    batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 


#CHANGE PARALLEL MODEL
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)


In [ ]:
print(pred[1])

In [ ]:
# arrange output of pred
min_x, min_y = 100000,100000
max_x, max_y = -1,-1

BLOCK_SIZE = 32
for predict in pred:
    #np arrange qualcosa con 49 x 5
    for el in arranged:
        if el[0] == 1:
            temp_width, temp_height = el[3]*BLOCK_SIZE, el[4]*BLOCK_SIZE,
            temp_lt_x, temp_lt_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            temp_rb_x, temp_rb_y = el[1]-temp_width/2, el[2]-temp_heigth/2
            if temp_lt_x<=min_x and temp_lt_y<=min_y:
                min_x, min_y= temp_lt_x,temp_lt_y
            else if temp_rb_x>=max_x and temp_rb_y>=max_y:
                max_x, max_y= temp_lt_x,temp_lt_y

print(min_x, min_y,max_x, max_y)

In [ ]:
from PIL import  Image, ImageDraw

count = 0

for gen_image in test_generator:
    img = Image.fromarray(np.uint8((gen_image[0][0])*255))
    img1 = ImageDraw.Draw(img)
    x1,y1 = pred[count][0],pred[count][1]
    x4,y4= pred[count][0] + pred[count][2], pred[count][1] + pred[count][3]
    img1.rectangle([(x1,y1),(x4,y4)], outline ="red") 
    #img.show() 
    img.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/NUOVO/results/output"+ str(count) + ".jpg")
    #print(gen_image[0][0].shape,pred[count])
    count = count + 1
    if count == 1000:
        break